In [ ]:
from types import SimpleNamespace

import numpy as np
from keras import backend as K
from keras.models import Model
from keras.layers import Activation, Conv2D, Input, UpSampling2D

from ipywidgets import interact
import matplotlib.pyplot as plt

## A Countable Model Class

We want a parameterized class of generative networks as models. The parameter values should such that, via a prefix-free encoding, they induce a _meaningful_ prior on the models. We desire a few properties:
- Every image of $N$ bits (uncompressed) can be generated by a model taking no input and having a prior probability as close to $2^{-N}$ as possible;
- There is a model taking $N$ bits of input such that every image of $N$ bits can be generated by it.

For convenience we may fix the size of the output image and pretend the size, $N$, is known to a model. To be completely thorough, this would add $\log N$ bits to the input of the model. The input of the model is treated as an enumeration of the images generated by it, so strictly speaking the model class should come with a way to scale models to arbitrary input sizes.

A large-ish image size limits the impact of small constants.

In [ ]:
# The size of the generated image
height = width = 256
channels = 1
## With color images (channels = 3), the lack of coherence between channels makes for very noisy images

# Let's go with 4 bits per pixel per channel (4096 colors)
uncompressed = height * width * channels * 4

In [ ]:
# We divide the uncompressed bits uniformly at random into three types of information
_splits = sorted(np.random.choice(uncompressed - 1, 2, replace=False))
soph = _splits[0] + 1
noise = _splits[1] - _splits[0]
redundancy = uncompressed - 1 - _splits[1]

print('uncompressed  ', uncompressed)
print('sophistication', soph)
print('noise         ', noise)
print('redundancy    ', redundancy)

We are now tasked with using `soph` bits to initiate a network that takes `noise` bits as input and generates `uncompressed` bits (shaped correctly) as output.
When choosing the `soph` bits and the `noise` bits randomly, then, with high probability, the generated image has a _sophistication_ and _noise_ close to `soph` and `noise`, respectively.

A typical convolutional neural network (CNN) architecture is `INPUT -> [[CONV -> RELU]*N -> POOL?]*M -> [FC -> RELU]*K -> FC` (http://cs231n.github.io/convolutional-networks/).
In order to generate images from unstructured data, we simply reverse this architecture.
Note that pooling layers are not invertible.
It has been argued that the pooling layers should be replaced by increasing the stride of the convolutions (https://arxiv.org/abs/1412.6806), which suggests a fractional stride for the reverse direction.
A fractional stride is implemented by padding each pixel with zeros (indicative of the loss of information in pooling).
However, a fractional stride executed like this can introduce artifacts (https://distill.pub/2016/deconv-checkerboard/).
Therefore, we choose to invert the pooling layers simply by upscaling.
Although ReLU activation is not invertible per se, no expressive power is lost by using ReLU activation for the reverse direction (but filter weights may need to be adjusted).

Our architecture comes with many hyperparameters that should ideally be parameters of the model class and hence part of the model encoding.
For simplicity, we shall fix them outside the model class and note that the total information represented by the parameters may be low in comparison to the sophistication.
The hyperparameters are

- The number of layers in the fully connected part and for each layer
  - The number of cells;
- The number of convolution blocks and for each block
  - The number of convolutions and for each convolution
    - The size of the filter (height, width, depth);

Upsampling happens after each block of convolutions and is conveniently done by a factor 2 in the height and width.

The hyperparameters determine the number of weights that must be encoded in the sophistication.
Typically, weights in a CNN are not distributed uniformly at random and a trained network can be compressed substantially.
We distinguish two approaches to compressing neural networks:
1. Invasive (e.g. pruning, [LeCun et al., 1989; Guo et al., 2016])
2. Non-invasive (e.g. quantization [Gupta et al., 2015])
The second approach comes in fixed-length and variable-length versions and differs in the amount of reproduction accuracy required accross the literature.
While variable-length and invasive techniques may produce better results, we choose to focus on fixed-length quantization.
If needed, it is possible to account for an additional `X%` improvement obtainable by different methods via an implicit `X/(1-X)%` increase in the available sophistication.

Experiments have shown that weights in a CNN typically follow a Laplace (two-sided exponential) distribution that has lower variance in layers further away from the image.
Quantizing with such a distribution, around 5 bits of accuracy are needed for acceptable performance (http://vijaychan.github.io/Publications/dcc-2017-cnn-compression.pdf).
However, in typical networks (e.g. AlexNet and VGG) the number of weights is dominated by the weights in the fully connected layer, for which our simple quantization scheme is of no use.
In deep convolutional networks (e.g. ResNet), most weights reside in convolutional layers.
We strive to keep the fully connected layer as small as possible, while still realizing the properties we desire of our model class.
In particular, when all information resides in the sophistication, we should resort to a simple fully connected layer where the weight encodes all information in a channel of a pixel (i.e. 4 bits).

It has been observed that CNNs with random filters are able to perform well.
This suggests that information can be pushed to the fully connected layer.
We shall not be too concerned with this and try to put the information in the convolutional layers.

From Wikipedia:
> **Number of filters**
>
> Since feature map size decreases with depth, layers near the input layer will tend to have fewer filters while higher layers can have more.
> To equalize computation at each layer, the feature x pixel position product is kept roughly constant across layers.
> Preserving more information about the input would require keeping the total number of activations (number of feature maps times number of pixel positions) non-decreasing from one layer to the next.

### Quick and Dirty

Let's ignore the fully connected layer for a moment.
There are roughly `soph // 5` weight variables available (drawn from Laplace distributions with varying variance), assuming 5 bits per weight.

The size of the output image is `height * width * channels`, whereas, ideally, the input is `1 * 1 * noise`.
Suppose we keep the kernel size fixed throughout the network (**`kernelsize`**) and let the depth of each convolution layer increases exponentially from the image towards the input (**`depthfactor`**), thus decreases with a factor `1 / depthfactor` towards the image.

This allows us to calculate the number of weights in the `i`th convolution layer from the image:

```kernelsize ** 2 * channels * depthfactor ** (i - 1) * channels * depthfactor ** i```

If their are `depth` convolution layers in the network, this amounts to a total number of weights of

```kernelsize ** 2 * channels ** 2 * depthfactor / (depthfactor ** 2 - 1) * [depthfactor ** (2 * depth) - 1]```

For convenience we match the depth of the input to `channels * depthfactor ** depth`, meaning that it may end up not being sized `1 * 1 * noise`.
Importantly, we must have `height / 2 ** depth <= sqrt(noise / (channels * depthfactor ** depth))`.
Additionally, we should demand `height >= sqrt(noise / (channels * depthfactor ** depth))` to hold.
This assumes we upsample at most every convolution layer and that we upsample with a factor of 2 in both height and width.

### Assumptions
- we keep the kernelsize constant throughout the network
- we distribute the upsampling steps evenly throughout the network
- the filterdepth decreases by a constant factor in every block

In [ ]:
def number_of_weights(kernelsize, depth, depthfactor):
    """Total number of weights in a network with the given hyperparameters"""
    try:
        return kernelsize**2 * channels**2 * depthfactor * (depthfactor**(2 * depth) - 1) / (depthfactor**2 - 1)
    except OverflowError:
        return float("Infinity")


def depth_to_factor(weights, kernelsize, depth):
    """Find a depthfactor corresponding to a depth given the number of weights"""
    # Test the degenerate limit case where depthfactor = 1
    if kernelsize**2 * channels**2 * depth > weights: return None
    PRECISION = 30
    low, high = (0.0, 1.0)
    for _ in range(PRECISION):
        candidate = (low + high) / 2
        candidate_weights = number_of_weights(kernelsize, depth, 1 / candidate)
        if candidate_weights < weights: high = candidate
        else: low = candidate

    # Last known depthfactor guaranteed to not generate too many weights
    return 1 / high


def possible_depths(noise, weights, kernelsize):
    """Generate all possible depths for a given number of weights and kernelsize"""
    depth = 1
    while True:
        depthfactor = depth_to_factor(weights, kernelsize, depth)
        if depthfactor is None: break
        reshapesize = np.sqrt(noise / (channels * depthfactor**depth))

        if height >= reshapesize and height / 2**depth <= reshapesize:
            yield (depth, depthfactor, reshapesize)
        depth += 1

### Hyperparameters

We pick some values for the model parameters ad-hoc and do not treat the length of their specification as part of the sophistication.
As the number of parameters is constant, this should not have a large effect on the specifics of the generated imagery.

In [ ]:
def adhoc_params(soph, noise, verbose=False):
    """We choose a low networkdepth so that the reshapesize and the spatial randomness are low"""
    params = SimpleNamespace(bpw = 5, kernelsize=7, depth=0, numblocks=0)
    for _depth, _depthfactor, _reshapesize in possible_depths(noise, soph / params.bpw, params.kernelsize):
        _numblocks = 1 + np.log2(height / _reshapesize)
        if verbose:
            print("Possible depth: {}\t(number of blocks: {:.3f})".format(_depth, _numblocks))
        if _numblocks >= params.numblocks:
            params.numblocks = int(_numblocks)
            params.depth = _depth
        elif not verbose: break
    if params.numblocks == 0: return None

    params.blockdepth = int(np.ceil(params.depth / params.numblocks))
    params.reshapesize = height // 2**(params.numblocks - 1)
    params.reshapedepth = max(int(noise / params.reshapesize**2), channels)
    params.depthfactor = (params.reshapedepth / channels)**(1 / params.depth)

    return params


params = adhoc_params(soph, noise, True)
print(params)

In [ ]:
def initializer_qlaplace(bpv, scale, verbose=False):
    """Construct a generator for elements from a quantized laplace distribution"""
    # ADHOC: We construct 2**bpv + 1 values, so that 0 is included
    quantiles = [scale * np.log(2 * (i + 1) / (2**bpv + 2)) for i in range(2**(bpv - 1))]
    if verbose:
        print("effective scaling (bpv={}, scale={}): {}".format(bpv, scale, np.prod(np.negative(quantiles))))
    quantiles = quantiles + [0.0] + [-q for q in quantiles]

    def sample(shape, dtype=None):
        return np.random.choice(quantiles, size=shape)
    return sample


def random_model(params, verbose=False):
    """Sample a model with the given hyperparameters"""
    kernel = (params.kernelsize, params.kernelsize)
    x = encoded = Input(shape=(params.reshapesize, params.reshapesize, params.reshapedepth))

    for block in range(params.numblocks - 1): # One too few
        # ADHOC: We let the variance decay as a function of the block index
        initializer = initializer_qlaplace(params.bpw,
                                           (1 + (block + 1) / params.numblocks) / params.kernelsize**2,
                                           verbose)
        for i in range(params.blockdepth):
            x = Conv2D(round(params.reshapedepth / params.depthfactor**(block * params.blockdepth + i + 1)),
                       kernel, activation='relu', padding='same', use_bias=False,
                       kernel_initializer=initializer)(x)
        x = UpSampling2D((2, 2))(x)
    # The remaining block (without upsampling)
    initializer = initializer_qlaplace(params.bpw, 2 / params.kernelsize**2, verbose)
    for i in range((params.numblocks - 1) * params.blockdepth, params.depth):
        x = Conv2D(round(params.reshapedepth / params.depthfactor**(i + 1)),
                   kernel, activation='relu', padding='same', use_bias=False,
                   kernel_initializer=initializer)(x)

    # Bring the outputs inside [0, 1] (TODO: 4 bits per channel discretization)
    decoded = Activation('sigmoid')(x)

    return Model(encoded, decoded)


if params is not None:
    m = random_model(params, True)
    m.summary()
    print("The number of parameters should be reasonably close to the number\n"
          "of weights ({:,.1f}) available from the sophistication.".format(soph / params.bpw))
else:
    print("No parameters associated with soph={} and noise={}.".format(soph, noise))

## Images

In [ ]:
n = 6

try:
    z = np.random.choice(2, (n, params.reshapesize, params.reshapesize, params.reshapedepth))
    y = m.predict(z, batch_size=n)
    if channels == 1: y = y.reshape([n, height, width])

    fig = plt.figure(figsize=(20, 20))
    for i in range(n):
        ax = fig.add_subplot(2, 3, i + 1, xticks=[], yticks=[])
        ax.imshow(y[i, ...])
    plt.show()
except NameError:
    print("No model")

## Now for some variation in parameters!

### Interactive

In [ ]:
def single_image(model):
    """Make some noise and generate a single image"""
    noise = np.random.choice(2, (1,) + K.int_shape(model.inputs[0])[1:])
    image = model.predict(noise, batch_size=1)[0, ...]
    if channels == 1: image = image.reshape([height, width])
    return image


@interact(compression=(0.0, 1.0), sophistication=(0.0, 1.0))
def single_show(compression=(soph+noise)/uncompressed, sophistication=soph/(soph+noise)):
    soph = int(sophistication * compression * uncompressed)
    noise = int(compression * uncompressed - soph + 0.5)
    plt.xticks([])
    plt.yticks([])
    try:
        plt.imshow(
            single_image(
                random_model(
                    adhoc_params(soph, noise)
                )
            )
        )
        plt.show()
    except Exception as e:
        print(e)

### Non-interactive (batch mode)

Noise increases rightward on the horizontal axis, while sophistication increases downward on the vertical axis.

In [ ]:
def multi_show(n, save='random-images.pdf'):
    step = (uncompressed - 2) // n
    fig = plt.figure(figsize=(20, 20))
    done = 0
    for soph in range(1, n):
        for noise in range(1, n - soph + 1):
            print("\r{} % ".format(int(100 * done / (n * (n - 1) / 2))), end='')
            ax = fig.add_subplot(n - 1, n - 1, (soph - 1) * (n - 1) + noise, xticks=[], yticks=[])
            try:
                ax.imshow(
                    single_image(
                        random_model(
                            adhoc_params(soph * step, noise * step)
                        )
                    )
                )
            except Exception as e:
                print(e)
            done += 1

    print("\rDone")
    if save is not None: plt.savefig(save)
    plt.show()


multi_show(16)